In [14]:
template = {
    'user_id': int,
    'name':{
        'first':str,
        'last':str
    },
    'bio':{
        'dob':{
            'year':int,
            'month':int,
            'day':int
        },
        'birthplace':{
            'country':str,
            'city':str
        }
    }
}

In [15]:
dean = {
    'user_id': 100,
    'name':{
        'first':'Dean',
        'last':'Winchester'
    },
    'bio':{
        'dob':{
            'year':1978,
            'month':11,
            'day':5
        },
        'birthplace':{
            'country':'USA',
        
        }
    }
}

In [16]:
sam = {
    'user_id': 102,
    'name':{
        'first':'Sam',
        'last':'Winchester'
    },
    'bio':{
        'dob':{
            'year':1982,
            'month':'May',
            'day':5
        },
        'birthplace':{
            'country':'USA',
            'city': 'San Antonio'
        
        }
    }
}

In [17]:
cass = {
    'user_id': 105,
    'name':{
        'first':'Cass',
        'last':'Heaven'
    },
    'bio':{
        'dob':{
            'year':1000,
            'month':1,
            'day':5
        },
        'birthplace':{
            'country':'Earth',
            'city': 'Heaven'
        
        }
    }
}

In [18]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()
    
    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys
    
    if missing_keys or extra_keys:
        missing_msg = ('missing keys: '+
                       ', '.join({path + '.'+str(key) 
                                  for key in missing_keys})
                      ) if missing_keys else ''
        extra_msg =  ('extra keys: '+
                      ', '.join({path + '.'+str(key) 
                                 for key in extra_keys})
                     ) if extra_keys else ''
        return False,' '.join((missing_msg,extra_msg))
    else:
        return True,None

In [19]:
t = {'a':int,'b':int,'c':int,'d':{}}
d= {'a': 'wrong type','b':100,'c':200,'d':{'wrong','type'}}
is_ok,err_msg = match_keys(d,t,'some.path')
print(is_ok,err_msg)

True None


In [20]:
t = {'a':int,'b':int,'c':int,'d':{}}
d = {'a':None,'b':None,'c':None}
is_ok,err_msg = match_keys(d,t,'some.path')
print(is_ok,err_msg)

False missing keys: some.path.d 


In [21]:
t = {'a':int,'b':int,'c':int,'d':{}}
d = {'a':None,'b':None,'c':None,'e':None}
is_ok,err_msg = match_keys(d,t,'some.path')
print(is_ok,err_msg)

False missing keys: some.path.d extra keys: some.path.e


In [22]:
t = {'a':int,'b':int,'c':int,'d':{}}
d = {'a':None,'b':None,'e':None,'f':None}
is_ok,err_msg = match_keys(d,t,'some.path')
print(is_ok,err_msg)

False missing keys: some.path.c, some.path.d extra keys: some.path.f, some.path.e


In [23]:
def match_types(data,template,path):
    for key,value in template.items():
        if isinstance (value,dict):
            template_type = dict
        else:
            template_type = value
        
        
        data_value = data.get(key,object())
        if not isinstance(data_value,template_type):
            err_msg = ('incorrect type: '+path+'.'+key+
                       '->expected' + template_type.__name__+
                       ',found '+type(data_value).__name__)
            return False, err_msg
    return True,None

In [24]:
t = {'a':int,'b':str,'c':{'d':int}}
d = {'a':100,'b':'test','c':{'some':'value'}}
match_types(d,t,'some.path')

(True, None)

In [25]:
t = {'a':int,'b':str,'c':{'d':int}}
d = {'a':100,'b':'test','c':'unexpected'}
match_types(d,t,'some.path')

(False, 'incorrect type: some.path.c->expecteddict,found str')

In [26]:
t = {'a':int,'b':str,'c':{'d':int}}
d = {'a':100,'b':200,'c':{}}
match_types(d,t,'some.path')

(False, 'incorrect type: some.path.b->expectedstr,found int')

In [27]:
def recurse_validate(data,template,path):
    is_ok,err_msg = match_keys(data,template,path)
    if not is_ok:
        return False,err_msg
    
    is_ok,err_msg = match_types(data,template,path)
    if not is_ok:
        return False,err_msg
    
    dictionary_type_keys = {key for key,value in template.items()
                           if isinstance(value,dict)}
    
    for key in dictionary_type_keys:
        sub_path = path +'.'+str(key)
        sub_template = template[key]
        sub_data = data[key]
        is_ok,err_msg = recurse_validate(sub_data,sub_template,sub_path)
        
        if not is_ok:
            return False, err_msg
        
    return True,None

In [28]:
is_ok, err_msg = recurse_validate(dean,template,'root')
print(is_ok, err_msg)


False missing keys: root.bio.birthplace.city 


In [29]:
is_ok, err_msg = recurse_validate(sam,template,'root')
print(is_ok, err_msg)


False incorrect type: root.bio.dob.month->expectedint,found str


In [30]:
def validate(data,template):
    return recurse_validate(data,template,'')

In [31]:
persons = ((dean,'Dean'),(sam,'Sam'),(cass,'Cass'))

In [32]:
for person,name in persons:
    is_ok,err_msg = validate(person,template)
    print(f'{name}:valide = {is_ok}:{err_msg}')

Dean:valide = False:missing keys: .bio.birthplace.city 
Sam:valide = False:incorrect type: .bio.dob.month->expectedint,found str
Cass:valide = True:None


In [33]:
class SchemaError(Exception):
    pass

In [34]:
def validate(data,template):
    is_ok,err_msg = recurse_validate(data,template, '')
    if not is_ok:
        raise SchemaError(err_msg)

In [35]:
try:
    for person,name in persons:
        validate(person,template)
except SchemaError as ex:
    print('Validation failed',str(ex))

Validation failed missing keys: .bio.birthplace.city 


In [36]:
validate(dean,template)

SchemaError: missing keys: .bio.birthplace.city 

In [37]:
class SchemaKeyMismatch(SchemaError):
    pass

class SchemaTypeMismatch(SchemaError, TypeError):
    pass

In [44]:
def match_keys(data, valid, path):
    data_keys = data.keys()
    valid_keys = valid.keys()
    
    extra_keys = data_keys - valid_keys
    missing_keys = valid_keys - data_keys
    
    if missing_keys or extra_keys:
        missing_msg = ('missing keys: '+
                       ', '.join({path + '.'+str(key) 
                                  for key in missing_keys})
                      ) if missing_keys else ''
        extra_msg =  ('extra keys: '+
                      ', '.join({path + '.'+str(key) 
                                 for key in extra_keys})
                     ) if extra_keys else ''
        raise SchemaTypeMismatch(' '.join((missing_msg,extra_msg)))


In [39]:
def match_types(data,template,path):
    for key,value in template.items():
        if isinstance (value,dict):
            template_type = dict
        else:
            template_type = value
        
        
        data_value = data.get(key,object())
        if not isinstance(data_value,template_type):
            err_msg = ('incorrect type: '+path+'.'+key+
                       '->expected' + template_type.__name__+
                       ',found '+type(data_value).__name__)
            raise SchemaTypeMismatch(err_msg)


In [40]:
def recurse_validate(data,template,path):
    match_keys(data,template,path)
    match_types(data,template,path)

    dictionary_type_keys = {key for key,value in template.items()
                           if isinstance(value,dict)}
    
    for key in dictionary_type_keys:
        sub_path = path +'.'+str(key)
        sub_template = template[key]
        sub_data = data[key]
        recurse_validate(sub_data,sub_template,sub_path)
        


In [41]:
def validate(data,template):
    recurse_validate(data,template, '')


In [42]:
validate(cass,template)

In [45]:
validate(dean,template)

SchemaTypeMismatch: missing keys: .bio.birthplace.city 

In [46]:
validate(sam,template)

SchemaTypeMismatch: incorrect type: .bio.dob.month->expectedint,found str

In [47]:
try:
    validate(sam,template)
except SchemaError as ex:
    print(ex)

incorrect type: .bio.dob.month->expectedint,found str


In [48]:
try:
    validate(dean,template)
except SchemaError as ex:
    print(ex)

missing keys: .bio.birthplace.city 


In [49]:
try:
    validate(cass,template)
except SchemaError as ex:
    print(ex)

In [53]:
try:
    validate(dean,template)
except SchemaKeyMismatch as ex:
    print('key mismatch')
except SchemaTypeMismatch as ex:
    print('type mismatch')
except SchemaError as ex:
    print('schema exception')
except TypeError as ex:
    print('type exception')

type mismatch
